In [1]:
import pandas as pd
import numpy as np

In [2]:
gradescope = pd.read_csv('gradescope.csv')

In [3]:
output = gradescope[['Name','SID','Email']]

In [4]:
cols = [i for i in gradescope.columns if 'Max Points' not in i]

In [5]:
gradescpe = gradescope[cols]

In [6]:
def determine_lateness(x,y):
    splitted =x.split(':')
    if int(splitted[0]) ==24 and int(splitted[1]) <= 5:
        return y*0.8
    elif int(splitted[0]) == 0 and int(splitted[1]) <= 5:
        return y
    elif int(splitted[0]) < 24:
        return y*0.8
    else:
        return y

In [7]:
def create_column(PA_col,with_checkpoint, with_part1,with_part2):
    PA = gradescope[[i for i in cols if PA_col in i]]
    PA_lateness = PA_col + ' - Lateness (H:M:S)'
    if 'final' in PA:
        grade_pa = PA[PA_col].fillna(0)
    else:
        grade_pa = PA.apply(lambda x: determine_lateness(x[PA_lateness],x[PA_col]),axis=1).fillna(0)
    if with_part1:
        grade_pa = grade_pa + PA.apply(lambda x: determine_lateness(x[PA_col+' Part 1 - Lateness (H:M:S)'],
                                                      x[PA_col+' Part 1']),axis=1).fillna(0)
    if with_part2:
        grade_pa = grade_pa + PA.apply(lambda x: determine_lateness(x[PA_col+' Part 2 - Lateness (H:M:S)'],
                                                      x[PA_col+' Part 2']),axis=1).fillna(0)
    if with_checkpoint:
        grade_pa = grade_pa+PA[PA_col+'_Checkpoint'].fillna(0)
    return grade_pa

In [9]:
PA_col = 'PA10' # Adjust column name here
# PA_col name (string), with_checkpoint(boolean), with_part1 (boolean), with_part2(boolean)
new_pa = create_column(PA_col,False,False,False) 
output[PA_col] = new_pa
output.tail(5)

/Users/elvychen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Name,SID,Email,PA9,PA10
178,Zhiyuan Yang,a15537584,z6yang@ucsd.edu,98.0,92.0
179,Zhuji Zhang,A16329547,zhz044@ucsd.edu,0.0,0.0
180,Zimin Dai,A14509602,z9dai@ucsd.edu,90.0,80.5
181,Zytal Lenus,A15398132,zlenus@ucsd.edu,0.0,0.0
182,mirodova@gmail.com,NaN,mirodova@gmail.com,0.0,0.0


In [35]:
midterms = []
for i in gradescope.columns:
    if 'Max Points' not in i and 'Submission' not in i and 'Lateness' not in i and 'self-proctor' not in i.lower():
        if 'Midterm' in i or 'midterm' in i:
            midterms.append(i)
midterms

['Midterm1', 'Midterm2']

In [36]:
output['Midterm2'] = gradescope['Midterm2'].fillna(0)

/Users/elvychen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
output.head()

,Name,SID,Email,PA9,PA10
0,Aishani Mohapatra,A16652568,aimohapatra@ucsd.edu,90.40,100.0
1,Akshat Muir,A16663537,akmuir@ucsd.edu,0.00,0.0
2,Alan Amirian,A15424176,aeamiria@ucsd.edu,79.00,83.0
3,Alec Mardikian,A15477624,amardiki@ucsd.edu,80.33,46.0
4,Alec Panattoni,A16562827,apanatto@ucsd.edu,103.00,91.0


In [11]:
output['SIS Login ID'] = output['Email'].apply(lambda x: x.split('@')[0])

/Users/elvychen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
output.columns

Index(['Name', 'SID', 'Email', 'PA9', 'PA10', 'SIS Login ID'], dtype='object')

In [13]:
canvas = pd.read_csv('canvas.csv')[['Student','ID','SIS User ID','SIS Login ID']]

In [14]:
canvas.head()

,Student,ID,SIS User ID,SIS Login ID
0,Points Possible,NaN,NaN,NaN
1,"Aguirre, Samuel",109581.0,A16687170,saguirre
2,"Ahmann, Nathan",63454.0,A15941151,nahmann
3,"Ahmed, Tonoya",108059.0,A16672415,ttahmed
4,"Ali, Rehan",101023.0,A16211966,rmali


In [15]:
output.head()

,Name,SID,Email,PA9,PA10,SIS Login ID
0,Aishani Mohapatra,A16652568,aimohapatra@ucsd.edu,90.40,100.0,aimohapatra
1,Akshat Muir,A16663537,akmuir@ucsd.edu,0.00,0.0,akmuir
2,Alan Amirian,A15424176,aeamiria@ucsd.edu,79.00,83.0,aeamiria
3,Alec Mardikian,A15477624,amardiki@ucsd.edu,80.33,46.0,amardiki
4,Alec Panattoni,A16562827,apanatto@ucsd.edu,103.00,91.0,apanatto


In [16]:
final = canvas.merge(output,left_on='SIS Login ID',right_on = 'SIS Login ID',how='left')
final.tail()

,Student,ID,SIS User ID,SIS Login ID,Name,SID,Email,PA9,PA10
146,"Yu, Zhaoyi",80694.0,A15901415,zhy003,Zhaoyi Yu,A15901415,zhy003@ucsd.edu,105.00,100.0
147,"Zavalza, Diego",94360.0,A16193208,dzavalza,Diego Zavalza,A16193208,dzavalza@ucsd.edu,75.00,46.5
148,"Zeng, Taylor",102480.0,A16379175,tczeng,Taylor Zeng,A16379175,tczeng@ucsd.edu,78.67,83.0
149,"Zhi, Xuzhe",99661.0,A16326028,xzhi,Xuzhe Zhi,A16326028,xzhi@ucsd.edu,98.00,100.0
150,"Student, Test",139419.0,NaN,9cd8f74bd5429f2668da4cd38814aaa3612ce07f,NaN,NaN,NaN,NaN,NaN


In [17]:
final.to_csv('final.csv') #copy grade columns into your canvas.csv and upload on canvas 